## Getting token for cds climate store

In [12]:
import psycopg2
import pandas.io.sql as sqlio
import getpass
import cdsapi
from fabric import Connection
from joblib import Parallel,delayed
import datetime
from tqdm import tqdm
import os
import uuid
import xarray as xr
import matplotlib.pyplot as plt
import json
from io import StringIO
import paramiko
import sys

sys.path.insert(0, "/home/jovyan/watexr/PROGNOS/")
from prognos_tools.gce_light import gce_api as gce

key = getpass.getpass('mobiserver password: ')
vaultKey = getpass.getpass('vault password: ')
#Querying necessary tokens
def query(query,fetch=True):
    with psycopg2.connect(user='jose-luis', host='mobiserver.niva.no', port=5432, database='vault',password=key) as db:
        with db.cursor() as cursor :
            cursor.execute(query)
            if fetch:
                result = sqlio.read_sql_query(query, db)
                return result
            
copernicusKey = query('''select niva.getToken('copernicusKey','{}');'''.format(vaultKey)).iloc[0,0]
cloudKey = json.loads(query('''select niva.getToken('gce_access','{}');'''.format(vaultKey)).iloc[0,0])
sshKey = query('''select niva.getToken('geonorway_ssh_key','{}');'''.format(vaultKey)).iloc[0,0]
not_really_a_file = StringIO(sshKey)
geo_key = paramiko.RSAKey.from_private_key(not_really_a_file)

del key,vaultKey,sshKey

with Connection('localhost') as c:
    c.local('rm -f /home/jovyan/.cdsapirc')
    c.local('touch /home/jovyan/.cdsapirc') 
    c.local('''echo "url: https://cds.climate.copernicus.eu/api/v2\n" >>/home/jovyan/.cdsapirc''')
    c.local('''echo "key: {}" >>/home/jovyan/.cdsapirc'''.format(copernicusKey))
    
del copernicusKey

mobiserver password:  ·······
vault password:  ···············


## Creating folder to store downloaded data on remote machine

In [13]:
#Properties of the instance containing the database
geo_props = {'project'      : 'nivacatchment',
             'zone'         : 'europe-north1-a',
             'instanceName' : "dtm10",
             'username'     : "jose-luis",
             }

#Setting up credential for google cloud
cloud = gce(geo_props, cloudKey)


#Getting instance info
cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
info = cloud.get('custom')
display(info['status'])
#If instance is stopped, start it
if info['status'] != 'RUNNING':
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}/start'''
    info = cloud.post('custom')
    display(info['status'])
    cloud.CommonCalls['custom'] = '''https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances/{instanceName}'''
    info = cloud.get('custom')
    while info['status'] != 'RUNNING':
           sleep(2)
           info = cloud.get('custom')
        
del cloudKey        
   
geo_ip = info['networkInterfaces'][0]['accessConfigs'][0]['natIP']    
display(geo_ip)
geo_config =  {'host' : geo_ip, 'user': 'jose-luis', 'connect_kwargs': {'pkey': geo_key } }


#Directory in the remote machine where the data will be placed
download_dir = '/home/jose-luis/era5'
with Connection(**geo_config) as c:
    c.run('rm -rf {0} && mkdir {0}'.format(download_dir))


'RUNNING'

'35.228.213.48'

## Querying data from the climate datastore

With the key obtained above, we can download data from the copernicus climate datastore. 
The available datasets are listed in https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset 

Let's try getting the potential evapotranspiration directly from the era5-land dataset: https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land?tab=overview

The ERA-5 dataset covers the whole world. We will only store data from Europe. It will be extracted from the (remote) global data using the fimex tool.  

In [ ]:
#Defining extent for Europe
#The .cfg file is used to pass the extent to the fimex tool
cfgName = 'extractEurope.cfg'   

cfgTemplate='''[extract]
reduceToBoundingBox.south= 33
reduceToBoundingBox.north= 82
reduceToBoundingBox.west= -14
reduceToBoundingBox.east= 35
'''

with open(cfgName, 'w') as f:
    f.write(cfgTemplate)
    
# Product and components to download.
# These are the variables that should be changed to access other datasets
#-----------------------------------------------------------------------------------------------------

    
components = ['10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
            '2m_temperature', 'clear_sky_direct_solar_radiation_at_surface', 'mean_sea_level_pressure',
            'mean_surface_downward_long_wave_radiation_flux_clear_sky', 'mean_surface_downward_short_wave_radiation_flux_clear_sky', 'potential_evaporation',
            'total_cloud_cover',
             ]
product = 'reanalysis-era5-single-levels'

hours = ['00:00','01:00','02:00',
'03:00','04:00','05:00',
'06:00','07:00','08:00',
'09:00','10:00','11:00',
'12:00','13:00','14:00',
'15:00','16:00','17:00',
'18:00','19:00','20:00',
'21:00','22:00','23:00'
]

area = [ 83, -15, 33, 36] #[90,-180,-90,180] for the world
#-----------------------------------------------------------------------------------------------------    
#The dataset needs to be postprocessed before it can be stored. Here we define the additional operations 
#To be run in the shell
def runInShell(operation,inFile):
    outFile = str(uuid.uuid4()) + '.nc'
    while True :
        with Connection('localhost') as c:
            if not os.path(outFile): 
                try :
                    c.local(operation.format(inFile=inFile,outFile=outFile),replace_env=False)
                except :
                    continue
    return outFile
                

#-----------------------------------------------------------------------------------------------------    
# Defining the functions to be applied to this particular dataset, by defaul none are
data_modifiers = ['''ncks -O --msa -d longitude,180.1,359.9 -d longitude,0.0,180.0 {inFile} {outFile}''',
                 '''ncap2 -O -s "where(longitude>180)longitude=round(longitude.double()*10.0-3600)/10.0" {inFile} {outFile}''',
                 '''fimex-1.5 -c {} --input.file {{}} --output.file {{}}'''.format(cfgName) 
                 ]


In [14]:
#Function to download the data, using the modifiers listed in the cell above   
#By default it DOES NOT overwrite if the output file exist
def getFile(inFile,date,outFile = [],data_modifiers=[]):
    #Downloading data
    file_to_test = inFile
    if outFile :
        file_to_test = outFile
    if not os.path.isfile(file_to_test):
        while True:
            if not os.path.isfile(inFile):
                try: 
                    d = cdsapi.Client()
                    d.retrieve(
                        product,
                        {
                             'product_type':'reanalysis',
                             'variable':components,
                            'year':  date.strftime("%Y"),
                            'month': date.strftime("%m"),
                            'day':   date.strftime("%d"),
                            'time': hours,
                            'area' : area,
                            'format':'netcdf'
                        },
                         inFile
                        )
                except:
                    with Connection('localhost') as c:
                        c.local('rm -f {}'.format(inFile))
                    continue
            #Postprocessing data        
            currentFile = inFile
            files_to_delete = []  #Temporary files create by the data modifier functions
            for i in data_modifiers:
                files_to_delete.append(currentFile)
                currentFile = runInShell(i,currentFile)
                
            #Moving resulting file and cleanup
            with Connection('localhost') as c:
                if os.path.isfile(currentFile):
                    try:
                        if files_to_delete:
                            c.local('rm -f {}'.format(' '.join(files_to_delete)))
                        if outFile :
                            with Connection(**geo_config) as f:
                                f.put(currentFile,outFile)
                            c.local('rm {}'.format(currentFile))
#                             c.local('mv {} {}'.format(currentFile,outFile))
                        break
                    except:
                        continue
            break

In [15]:
#Setting dates to download data

start = datetime.datetime.strptime("01-01-1979", "%d-%m-%Y")
end = datetime.datetime.strptime("08-01-1979", "%d-%m-%Y")
# end = datetime.datetime.strptime("20-03-2021", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

#The data will be downloaded in parallel in split int the following number of workers (rule of thumb: twice as many workers as processors)
max_workers = 2
num_files = len(date_generated)

#Creating directory to download data
# download_dir = '/home/jovyan/shared/era5'
# with Connection('localhost') as c:
#     c.local('rm -rf {}'.format(download_dir))
#     c.local('mkdir {}'.format(download_dir))   

#Download function: the date is included in the generated filename   
def getData(date):    
#     print(date.strftime('%d-%m-%Y'))
    filename = '{}.nc'.format(date.strftime('%d-%m-%Y'))
    out =  '{}/europe_{}.nc'.format(download_dir,date.strftime('%d-%m-%Y'))
    getFile(filename,date,outFile=out) 

In [17]:
#Testing the download for a single date
with Connection('localhost') as c:
    c.local('rm -f world.nc')
    
date = datetime.datetime(1990,5,5)
getFile('world.nc',date)

data = xr.load_dataset('world.nc')
v10 = data['v10'][0,:,:]
plt.figure(figsize = (20,10))
plt.imshow(v10.values)

ValueError: I/O operation on closed file

In [16]:
# %%capture
#Actually downloading the data for all the dates of interest
Parallel(n_jobs=max_workers)(delayed(getData)(date) for date in tqdm(date_generated))

100%|██████████| 1/1 [00:00<00:00, 26.46it/s]


PicklingError: Could not pickle the task to send it to the workers.

## Check all files have been downloaded

In [ ]:
%%capture
#Checking if all the files in the list of dates have been downloaded
from parse import *

#Getting the dates that have been downloaded from the file name
with Connection('localhost') as c:
    a = c.local('cd {} && ls -1 *.nc'.format(download_dir,hide='stdout'))
#     display(a)                
a = a.stdout.split('\n')
a = [ parse('europe_{d}-{m}-{Y}.nc',i) for i in a]
a = [i.named for i in a  if i is not None]
a = [datetime.datetime(int(i['Y']),int(i['m']),int(i['d'])) for i in a]


#Checking the difference between the downloaded files and the list of dates:
missing_files = list(set(date_generated) - set(a))
missing_files.sort()


In [ ]:
display(len(missing_files))

In [ ]:
import pickle
with open('missing.pickle','wb') as f:
    pickle.dump(missing_files,f,protocol=pickle.HIGHEST_PROTOCOL)
    
with open('missing.pickle','rb') as f:
    bla = pickle.load(f)
    
display(len(bla))

In [ ]:
config

In [18]:
with Connection(**geo_config) as c:
    c.put('/home/jovyan/.cdsapirc','/home/jose-luis/')


2021-03-22 11:32:58,274 INFO Connected (version 2.0, client OpenSSH_7.9p1)
2021-03-22 11:32:58,382 INFO Authentication (publickey) successful!
2021-03-22 11:32:58,447 INFO [chan 0] Opened sftp connection (server version 3)


In [23]:
%%writefile getEra.py
#! /usr/bin/python3
#This script should be run in the geodatabase vm. It downloads era5 data
import psycopg2
import pandas.io.sql as sqlio
import getpass
import cdsapi
from fabric import Connection
from joblib import Parallel,delayed
import datetime
from tqdm import tqdm
import os
import uuid
import xarray as xr

#Defining extent for Europe
#The .cfg file is used to pass the extent to the fimex tool
cfgName = 'extractEurope.cfg'   

cfgTemplate='''[extract]
reduceToBoundingBox.south= 33
reduceToBoundingBox.north= 82
reduceToBoundingBox.west= -14
reduceToBoundingBox.east= 35
'''

with open(cfgName, 'w') as f:
    f.write(cfgTemplate)
    
# Product and components to download.
# These are the variables that should be changed to access other datasets
#-----------------------------------------------------------------------------------------------------

    
components = ['10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
            '2m_temperature', 'clear_sky_direct_solar_radiation_at_surface', 'mean_sea_level_pressure',
            'mean_surface_downward_long_wave_radiation_flux_clear_sky', 'mean_surface_downward_short_wave_radiation_flux_clear_sky', 'potential_evaporation',
            'total_cloud_cover',
             ]
product = 'reanalysis-era5-single-levels'

hours = ['00:00','01:00','02:00',
'03:00','04:00','05:00',
'06:00','07:00','08:00',
'09:00','10:00','11:00',
'12:00','13:00','14:00',
'15:00','16:00','17:00',
'18:00','19:00','20:00',
'21:00','22:00','23:00'
]

area = [ 83, -15, 33, 36] #[90,-180,-90,180] for the world
#-----------------------------------------------------------------------------------------------------    
#The dataset needs to be postprocessed before it can be stored. Here we define the additional operations 
#To be run in the shell
def runInShell(operation,inFile):
    outFile = str(uuid.uuid4()) + '.nc'
    while True :
        with Connection('localhost') as c:
            if not os.path(outFile): 
                try :
                    c.local(operation.format(inFile=inFile,outFile=outFile),replace_env=False)
                except :
                    continue
    return outFile
                

#-----------------------------------------------------------------------------------------------------    
# Defining the functions to be applied to this particular dataset, by defaul none are
data_modifiers = ['''ncks -O --msa -d longitude,180.1,359.9 -d longitude,0.0,180.0 {inFile} {outFile}''',
                 '''ncap2 -O -s "where(longitude>180)longitude=round(longitude.double()*10.0-3600)/10.0" {inFile} {outFile}''',
                 '''fimex-1.5 -c {} --input.file {{}} --output.file {{}}'''.format(cfgName) 
                 ]

#-----------------------------------------------------------------------------------------------------    
#Function to download the data, using the modifiers listed in the cell above   
#By default it DOES NOT overwrite if the output file exist
def getFile(inFile,date,outFile = [],data_modifiers=[]):
    #Downloading data
    file_to_test = inFile
    if outFile :
        file_to_test = outFile
    if not os.path.isfile(file_to_test):
        while True:
            if not os.path.isfile(inFile):
                try: 
                    d = cdsapi.Client()
                    d.retrieve(
                        product,
                        {
                             'product_type':'reanalysis',
                             'variable':components,
                            'year':  date.strftime("%Y"),
                            'month': date.strftime("%m"),
                            'day':   date.strftime("%d"),
                            'time': hours,
                            'area' : area,
                            'format':'netcdf'
                        },
                         inFile
                        )
                except:
                    with Connection('localhost') as c:
                        c.local('rm -f {}'.format(inFile))
                    continue
            #Postprocessing data        
            currentFile = inFile
            files_to_delete = []  #Temporary files create by the data modifier functions
            for i in data_modifiers:
                files_to_delete.append(currentFile)
                currentFile = runInShell(i,currentFile)
                
            #Moving resulting file and cleanup
            with Connection('localhost') as c:
                if os.path.isfile(currentFile):
                    try:
                        if files_to_delete:
                            c.local('rm -f {}'.format(' '.join(files_to_delete)))
                        c.local('mv {} {}'.format(currentFile,outFile))
                        break
                    except:
                        continue
            break
            
#-----------------------------------------------------------------------------------------------------    
#Setting dates to download data
start = datetime.datetime.strptime("01-01-1979", "%d-%m-%Y")
end = datetime.datetime.strptime("08-01-1979", "%d-%m-%Y")
# end = datetime.datetime.strptime("20-03-2021", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

#The data will be downloaded in parallel in split int the following number of workers (rule of thumb: twice as many workers as processors)
max_workers = 2
num_files = len(date_generated)

#Creating directory to download data
download_dir = '/home/jose-luis/era5'
with Connection('localhost') as c:
    c.local('rm -rf {}'.format(download_dir))
    c.local('mkdir {}'.format(download_dir))   

#Download function: the date is included in the generated filename   
def getData(date):    
#     print(date.strftime('%d-%m-%Y'))
    filename = '{}.nc'.format(date.strftime('%d-%m-%Y'))
    out =  '{}/europe_{}.nc'.format(download_dir,date.strftime('%d-%m-%Y'))
    getFile(filename,date,outFile=out) 
    
#-----------------------------------------------------------------------------------------------------  
Parallel(n_jobs=max_workers)(delayed(getData)(date) for date in tqdm(date_generated))

Overwriting getEra.py


In [24]:
with Connection(**geo_config) as c:
    c.put('getEra.py','/home/jose-luis/')
    c.run('chmod +x getEra.py')

2021-03-22 11:34:46,995 INFO Connected (version 2.0, client OpenSSH_7.9p1)
2021-03-22 11:34:47,101 INFO Authentication (publickey) successful!
2021-03-22 11:34:47,162 INFO [chan 0] Opened sftp connection (server version 3)
